![Team USA Football](images/Usa_football_body_logo.png)

Data Dictionary here https://nflreadr.nflverse.com/articles/dictionary_player_stats.html

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.dummy import DummyRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split , cross_val_score
from sklearn.metrics import mean_squared_error

ill save my data as some new dataframes and see what i have.

In [2]:
stats_df = pd.read_csv('data/player_stats.csv')

ill start with stats_df as that will require more work

In [3]:
stats_df.head()

,player_id,player_name,player_display_name,position,position_group,headshot_url,recent_team,season,week,season_type,...,receiving_first_downs,receiving_epa,receiving_2pt_conversions,racr,target_share,air_yards_share,wopr,special_teams_tds,fantasy_points,fantasy_points_ppr
0,00-0000003,NaN,Abdul-Karim al-Jabbar,RB,RB,NaN,MIA,1999,1,REG,...,0,0.292378,0,0.0,0.052632,NaN,NaN,0,12.7,13.7
1,00-0000003,NaN,Abdul-Karim al-Jabbar,RB,RB,NaN,MIA,1999,2,REG,...,1,0.377009,0,0.0,0.117647,NaN,NaN,0,5.1,8.1
2,00-0000003,NaN,Abdul-Karim al-Jabbar,RB,RB,NaN,MIA,1999,4,REG,...,0,-0.699578,0,NaN,0.023810,NaN,NaN,0,0.2,0.2
3,00-0000003,NaN,Abdul-Karim al-Jabbar,RB,RB,NaN,CLE,1999,7,REG,...,0,-0.228454,0,0.0,0.050000,NaN,NaN,0,3.5,5.5
4,00-0000003,NaN,Abdul-Karim al-Jabbar,RB,RB,NaN,CLE,1999,8,REG,...,0,NaN,0,NaN,NaN,NaN,NaN,0,3.9,3.9


In [4]:
# lets check up on Aaron Rodgers
stats_df[stats_df['player_display_name'] == 'Aaron Rodgers'].head()

,player_id,player_name,player_display_name,position,position_group,headshot_url,recent_team,season,week,season_type,...,receiving_first_downs,receiving_epa,receiving_2pt_conversions,racr,target_share,air_yards_share,wopr,special_teams_tds,fantasy_points,fantasy_points_ppr
34491,00-0023459,A.Rodgers,Aaron Rodgers,QB,QB,https://static.www.nfl.com/image/private/f_aut...,GB,2005,5,REG,...,0,NaN,0,NaN,NaN,NaN,NaN,0,0.00,0.00
34492,00-0023459,A.Rodgers,Aaron Rodgers,QB,QB,https://static.www.nfl.com/image/private/f_aut...,GB,2005,15,REG,...,0,NaN,0,NaN,NaN,NaN,NaN,0,-2.60,-2.60
34493,00-0023459,A.Rodgers,Aaron Rodgers,QB,QB,https://static.www.nfl.com/image/private/f_aut...,GB,2005,17,REG,...,0,NaN,0,NaN,NaN,NaN,NaN,0,-0.10,-0.10
38709,00-0023459,A.Rodgers,Aaron Rodgers,QB,QB,https://static.www.nfl.com/image/private/f_aut...,GB,2006,4,REG,...,0,NaN,0,NaN,NaN,NaN,NaN,0,0.56,0.56
38710,00-0023459,A.Rodgers,Aaron Rodgers,QB,QB,https://static.www.nfl.com/image/private/f_aut...,GB,2006,11,REG,...,0,NaN,0,NaN,NaN,NaN,NaN,0,0.38,0.38


In [5]:
stats_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 129739 entries, 0 to 129738
Data columns (total 53 columns):
 #   Column                       Non-Null Count   Dtype  
---  ------                       --------------   -----  
 0   player_id                    129739 non-null  object 
 1   player_name                  61504 non-null   object 
 2   player_display_name          129736 non-null  object 
 3   position                     129667 non-null  object 
 4   position_group               129667 non-null  object 
 5   headshot_url                 69803 non-null   object 
 6   recent_team                  129739 non-null  object 
 7   season                       129739 non-null  int64  
 8   week                         129739 non-null  int64  
 9   season_type                  129739 non-null  object 
 10  opponent_team                129739 non-null  object 
 11  completions                  129739 non-null  int64  
 12  attempts                     129739 non-null  int64  
 13 

In [6]:
# Use 1 to show all info in cell below then comment it out and use 2 to go back to default view I can delete this later

#1
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', None)

#2
#pd.reset_option('display.max_rows')
#pd.reset_option('display.max_columns')
#pd.reset_option('display.max_colwidth')


In [7]:
stats_df['position'].value_counts()

WR     49363
RB     33350
TE     24147
QB     16086
FB      5219
P        459
CB       148
G        128
DB       123
T         94
FS        93
SS        77
OLB       56
LS        56
K         46
LB        45
ILB       32
C         28
DE        28
DT        20
OT        17
DL        15
HB        13
NT        10
MLB        8
OG         3
OL         2
S          1
Name: position, dtype: int64

ive only got offensive related stats going back to 1999. I wont be needing those older players in a team for today

In [8]:
#lets look at the last ten years of data 
stats_df = stats_df[stats_df['season'].isin(range(2014, 2024))]

In [9]:
#lets only look at the regular season so we can see every player in action. this is about players not teams
stats_df = stats_df[stats_df['season_type'] == 'REG']


again these are offensive stats but they do give me a few key pieces of info that i can credit the oppenent team with. the opponent of an offense being.....a defense!!

# Creating Defense 

In [10]:
# Define a function to handle grouping, summing, renaming, and filling NaNs
def process_stat(stat_df, stat_name, new_name=None):
    if new_name is None:
        new_name = stat_name
    grouped_data = stat_df.groupby(['opponent_team', 'season'])[stat_name].sum().reset_index()
    grouped_data.rename(columns={'opponent_team': 'defense', stat_name: new_name}, inplace=True)
    grouped_data.fillna(0, inplace=True)
    return grouped_data

# List of statistics to process
stats_to_process = [
    ('interceptions', 'interceptions_total'),
    ('sacks', 'sacks'),
    ('sack_yards', 'sack_yards'),
    ('receiving_yards', 'receiving_yards'),
    ('rushing_yards', 'rushing_yards'),
    ('rushing_tds', 'rushing_tds'),
    ('receiving_tds', 'receiving_tds'),
    ('sack_fumbles', 'sack_fumbles'),
    ('rushing_fumbles', 'rushing_fumbles'),
    ('receiving_fumbles', 'receiving_fumbles'),
    ('sack_fumbles_lost', 'sack_fumbles_recovered'),
    ('rushing_fumbles_lost', 'rushing_fumbles_recovered'),
    ('receiving_fumbles_lost', 'receiving_fumbles_recovered')
]

# Process each stat and store in a list of DataFrames
processed_stats = [process_stat(stats_df, stat, new_name) for stat, new_name in stats_to_process]

# Merge all DataFrames on 'defense' and 'season'
defense_df = processed_stats[0]
for df in processed_stats[1:]:
    defense_df = pd.merge(defense_df, df, on=['defense', 'season'], how='outer')

# Display the resulting DataFrame
defense_df.head()

,defense,season,interceptions_total,sacks,sack_yards,receiving_yards,rushing_yards,rushing_tds,receiving_tds,sack_fumbles,rushing_fumbles,receiving_fumbles,sack_fumbles_recovered,rushing_fumbles_recovered,receiving_fumbles_recovered
0,ARI,2014,18,35,244,4396,1739,9,22,2,4,4,1,1,4
1,ARI,2015,19,36,270,3957,1460,9,24,8,12,4,5,5,2
2,ARI,2016,14,48,374,3739,1518,16,21,12,11,2,6,4,1
3,ARI,2017,15,37,255,3796,1434,12,24,7,7,3,2,1,2
4,ARI,2018,7,49,333,3595,2479,25,21,7,3,5,6,1,2


In [11]:
defense_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 320 entries, 0 to 319
Data columns (total 15 columns):
 #   Column                       Non-Null Count  Dtype 
---  ------                       --------------  ----- 
 0   defense                      320 non-null    object
 1   season                       320 non-null    int64 
 2   interceptions_total          320 non-null    int64 
 3   sacks                        320 non-null    int64 
 4   sack_yards                   320 non-null    int64 
 5   receiving_yards              320 non-null    int64 
 6   rushing_yards                320 non-null    int64 
 7   rushing_tds                  320 non-null    int64 
 8   receiving_tds                320 non-null    int64 
 9   sack_fumbles                 320 non-null    int64 
 10  rushing_fumbles              320 non-null    int64 
 11  receiving_fumbles            320 non-null    int64 
 12  sack_fumbles_recovered       320 non-null    int64 
 13  rushing_fumbles_recovered    320 no

just need to create some ratios and totals for defense and that should be ready 

In [12]:
# Create forced_fumbles column
defense_df['forced_fumbles'] = (defense_df['sack_fumbles'] +
                                defense_df['rushing_fumbles'] +
                                defense_df['receiving_fumbles']
                               )
# Create fumbles_recovered column
defense_df['fumbles_recovered'] = (defense_df['sack_fumbles_recovered'] +
                                   defense_df['rushing_fumbles_recovered'] +
                                   defense_df['receiving_fumbles_recovered']
                                  )

# Calculate the percentage of forced fumbles that were fumbles recovered
defense_df['percentage_fumbles_recovered'] = (defense_df['fumbles_recovered'] / defense_df['forced_fumbles'] * 100).fillna(0)

# Create tds_against column
defense_df['tds_against'] = (defense_df['rushing_tds'] +
                             defense_df['receiving_tds']
                            )

#Create yards_against column
defense_df['yards_against'] = (defense_df['rushing_yards'] +
                             defense_df['receiving_yards'] -
                             defense_df['sack_yards']
                            )

# Display the updated DataFrame
defense_df.head()

,defense,season,interceptions_total,sacks,sack_yards,receiving_yards,rushing_yards,rushing_tds,receiving_tds,sack_fumbles,rushing_fumbles,receiving_fumbles,sack_fumbles_recovered,rushing_fumbles_recovered,receiving_fumbles_recovered,forced_fumbles,fumbles_recovered,percentage_fumbles_recovered,tds_against,yards_against
0,ARI,2014,18,35,244,4396,1739,9,22,2,4,4,1,1,4,10,6,60.000000,31,5891
1,ARI,2015,19,36,270,3957,1460,9,24,8,12,4,5,5,2,24,12,50.000000,33,5147
2,ARI,2016,14,48,374,3739,1518,16,21,12,11,2,6,4,1,25,11,44.000000,37,4883
3,ARI,2017,15,37,255,3796,1434,12,24,7,7,3,2,1,2,17,5,29.411765,36,4975
4,ARI,2018,7,49,333,3595,2479,25,21,7,3,5,6,1,2,15,9,60.000000,46,5741


In [13]:
defense_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 320 entries, 0 to 319
Data columns (total 20 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   defense                       320 non-null    object 
 1   season                        320 non-null    int64  
 2   interceptions_total           320 non-null    int64  
 3   sacks                         320 non-null    int64  
 4   sack_yards                    320 non-null    int64  
 5   receiving_yards               320 non-null    int64  
 6   rushing_yards                 320 non-null    int64  
 7   rushing_tds                   320 non-null    int64  
 8   receiving_tds                 320 non-null    int64  
 9   sack_fumbles                  320 non-null    int64  
 10  rushing_fumbles               320 non-null    int64  
 11  receiving_fumbles             320 non-null    int64  
 12  sack_fumbles_recovered        320 non-null    int64  
 13  rushi

In [69]:
defense_df.head()

,defense,season,interceptions_total,sacks,sack_yards,receiving_yards,rushing_yards,rushing_tds,receiving_tds,sack_fumbles,rushing_fumbles,receiving_fumbles,sack_fumbles_recovered,rushing_fumbles_recovered,receiving_fumbles_recovered,forced_fumbles,fumbles_recovered,percentage_fumbles_recovered,tds_against,yards_against
0,ARI,2014,18,35,244,4396,1739,9,22,2,4,4,1,1,4,10,6,60.000000,31,5891
1,ARI,2015,19,36,270,3957,1460,9,24,8,12,4,5,5,2,24,12,50.000000,33,5147
2,ARI,2016,14,48,374,3739,1518,16,21,12,11,2,6,4,1,25,11,44.000000,37,4883
3,ARI,2017,15,37,255,3796,1434,12,24,7,7,3,2,1,2,17,5,29.411765,36,4975
4,ARI,2018,7,49,333,3595,2479,25,21,7,3,5,6,1,2,15,9,60.000000,46,5741


these stats dont tell the full story, i dont know if a defense scored, i only know that they were scored on so lets use that and any other stat that our analysis dictates as useful 

In [71]:
cleaned_defense_df = defense_df[['defense', 'season' ,'tds_against', 'rushing_tds', 'receiving_tds', 'receiving_yards', 'rushing_yards']]
cleaned_defense_df.head()

,defense,season,tds_against,rushing_tds,receiving_tds,receiving_yards,rushing_yards
0,ARI,2014,31,9,22,4396,1739
1,ARI,2015,33,9,24,3957,1460
2,ARI,2016,37,16,21,3739,1518
3,ARI,2017,36,12,24,3796,1434
4,ARI,2018,46,25,21,3595,2479


i dont think i use any statistical models for the defense. ill make visualizations here later 

# The Offense

In [15]:
#now lets move back to the offense and seperate by position for our 3 analyses
# Filter for quarterbacks
qbs_df = stats_df[stats_df['position'] == 'QB']

# Filter for rushers (RBs and FBs)
rushers_df = stats_df[stats_df['position'].isin(['RB', 'FB', 'HB'])]

# Filter for receivers (WRs and TEs)
receivers_df = stats_df[stats_df['position'].isin(['WR', 'TE'])]

In [16]:
# Create efficiency metrics
qbs_df.loc[:, 'completion_percentage'] = qbs_df['completions'] / qbs_df['attempts']
qbs_df.loc[:, 'yards_per_attempt'] = qbs_df['passing_yards'] / qbs_df['attempts']
rushers_df.loc[:, 'yards_per_carry'] = rushers_df['rushing_yards'] / rushers_df['carries']
receivers_df.loc[:, 'yards_per_reception'] = receivers_df['receiving_yards'] / receivers_df['receptions']

# Create touchdown ratios
qbs_df.loc[:, 'pass_tds_per_attempt'] = qbs_df['passing_tds'] / qbs_df['attempts']
rushers_df.loc[:, 'tds_per_carry'] = rushers_df['rushing_tds'] / rushers_df['carries']
receivers_df.loc[:, 'tds_per_reception'] = receivers_df['receiving_tds'] / receivers_df['receptions']

# Calculate season stats for each player over the season
qbs_df.loc[:, 'season_passing_yards'] = qbs_df.groupby(['player_id', 'season'])['passing_yards'].transform('sum')
rushers_df.loc[:, 'season_rushing_yards'] = rushers_df.groupby(['player_id', 'season'])['rushing_yards'].transform('sum')
rushers_df.loc[:, 'season_carries'] = rushers_df.groupby(['player_id', 'season'])['carries'].transform('sum')
receivers_df.loc[:, 'season_receiving_yards'] = receivers_df.groupby(['player_id', 'season'])['receiving_yards'].transform('sum')
qbs_df.loc[:, 'season_passing_tds'] = qbs_df.groupby(['player_id', 'season'])['passing_tds'].transform('sum')
rushers_df.loc[:, 'season_rushing_tds'] = rushers_df.groupby(['player_id', 'season'])['rushing_tds'].transform('sum')
receivers_df.loc[:, 'season_receiving_tds'] = receivers_df.groupby(['player_id', 'season'])['receiving_tds'].transform('sum')
receivers_df.loc[:, 'season_receptions'] = receivers_df.groupby(['player_id', 'season'])['receptions'].transform('sum')
qbs_df.loc[:, 'season_pass_attempts'] = qbs_df.groupby(['player_id', 'season'])['attempts'].transform('sum')
qbs_df.loc[:, 'season_completions'] = qbs_df.groupby(['player_id', 'season'])['completions'].transform('sum')
qbs_df.loc[:, 'season_completion_percentage'] = qbs_df.groupby(['player_id', 'season'])['completion_percentage'].transform('mean')
qbs_df.loc[:, 'season_yards_per_attempt'] = qbs_df.groupby(['player_id', 'season'])['yards_per_attempt'].transform('mean')
qbs_df.loc[:, 'season_pass_tds_per_attempt'] = qbs_df.groupby(['player_id', 'season'])['pass_tds_per_attempt'].transform('mean')
rushers_df.loc[:, 'season_yards_per_carry'] = rushers_df.groupby(['player_id', 'season'])['yards_per_carry'].transform('mean')
receivers_df.loc[:, 'season_yards_per_reception'] = receivers_df.groupby(['player_id', 'season'])['yards_per_reception'].transform('mean')
rushers_df.loc[:, 'season_tds_per_carry'] = rushers_df.groupby(['player_id', 'season'])['tds_per_carry'].transform('mean')
receivers_df.loc[:, 'season_tds_per_reception'] = receivers_df.groupby(['player_id', 'season'])['tds_per_reception'].transform('mean')

c:\Users\Trugrit\anaconda3\envs\learn-env\lib\site-packages\pandas\core\indexing.py:1596: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = _infer_fill_value(value)
c:\Users\Trugrit\anaconda3\envs\learn-env\lib\site-packages\pandas\core\indexing.py:1745: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(ilocs[0], value)


## Drop unnesscessary columns from each df

In [17]:
qbs_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 6211 entries, 75507 to 129722
Data columns (total 63 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   player_id                     6211 non-null   object 
 1   player_name                   5993 non-null   object 
 2   player_display_name           6211 non-null   object 
 3   position                      6211 non-null   object 
 4   position_group                6211 non-null   object 
 5   headshot_url                  6154 non-null   object 
 6   recent_team                   6211 non-null   object 
 7   season                        6211 non-null   int64  
 8   week                          6211 non-null   int64  
 9   season_type                   6211 non-null   object 
 10  opponent_team                 6211 non-null   object 
 11  completions                   6211 non-null   int64  
 12  attempts                      6211 non-null   int64  
 1

In [18]:
qbs_df = qbs_df.drop(columns= ['player_id',
                               'player_name',
                               'position', 
                               'position_group',
                               'headshot_url',
                               'season_type',   
                               'passing_epa',
                               'pacr',
                               'dakota',
                               'attempts',
                               'completions',
                               'passing_yards',
                               'passing_tds',
                               'interceptions',
                               'sacks',
                               'sack_yards',
                               'sack_fumbles',
                               'sack_fumbles_lost',
                               'passing_air_yards',
                               'passing_yards_after_catch',
                               'passing_first_downs',
                               'passing_epa',
                               'passing_2pt_conversions',
                               'carries',
                               'rushing_yards',
                               'rushing_tds',
                               'rushing_fumbles',
                               'rushing_fumbles_lost',
                               'rushing_first_downs',
                               'rushing_epa',
                               'rushing_2pt_conversions',
                               'receptions',
                               'targets',
                               'receiving_yards',
                               'receiving_tds',
                               'receiving_fumbles',
                               'receiving_fumbles_lost',
                               'receiving_air_yards',
                               'receiving_yards_after_catch',
                               'receiving_first_downs',
                               'receiving_epa',
                               'receiving_2pt_conversions',
                               'racr',
                               'target_share',
                               'air_yards_share',
                               'wopr',
                               'special_teams_tds',
                               'fantasy_points',
                               'fantasy_points_ppr'
                              ]).reset_index(drop=True)
qbs_df.head()

,player_display_name,recent_team,season,week,opponent_team,completion_percentage,yards_per_attempt,pass_tds_per_attempt,season_passing_yards,season_passing_tds,season_pass_attempts,season_completions,season_completion_percentage,season_yards_per_attempt,season_pass_tds_per_attempt
0,Matt Hasselbeck,IND,2014,3,JAX,0.500000,5.000000,0.000000,301,2,44,30,0.632832,6.385965,0.033417
1,Matt Hasselbeck,IND,2014,4,TEN,NaN,NaN,NaN,301,2,44,30,0.632832,6.385965,0.033417
2,Matt Hasselbeck,IND,2014,16,DAL,0.714286,6.000000,0.047619,301,2,44,30,0.632832,6.385965,0.033417
3,Matt Hasselbeck,IND,2014,17,TEN,0.684211,8.157895,0.052632,301,2,44,30,0.632832,6.385965,0.033417
4,Peyton Manning,DEN,2014,1,IND,0.611111,7.472222,0.083333,4727,39,597,395,0.673296,8.169275,0.068182


In [19]:
# Get the index of the maximum week for each player
max_week_idx = qbs_df.groupby(['player_display_name', 'season'])['week'].idxmax()

# Select the rows corresponding to these indices
qbs_df = qbs_df.loc[max_week_idx].reset_index(drop=True)
qbs_df = qbs_df.drop(columns= 'week')
qbs_df.head()

,player_display_name,recent_team,season,opponent_team,completion_percentage,yards_per_attempt,pass_tds_per_attempt,season_passing_yards,season_passing_tds,season_pass_attempts,season_completions,season_completion_percentage,season_yards_per_attempt,season_pass_tds_per_attempt
0,AJ McCarron,CIN,2015,BAL,0.629630,5.925926,0.074074,854,6,119,79,0.721109,6.477734,0.035461
1,AJ McCarron,CIN,2017,MIN,0.500000,3.166667,0.000000,66,0,14,7,0.500000,4.520833,0.000000
2,AJ McCarron,LV,2018,BAL,NaN,NaN,NaN,8,0,3,1,0.333333,2.666667,0.000000
3,AJ McCarron,HOU,2019,TEN,0.583333,6.250000,0.000000,225,0,37,21,0.291667,3.125000,0.000000
4,AJ McCarron,HOU,2020,TEN,1.000000,20.000000,0.000000,20,0,1,1,1.000000,20.000000,0.000000


In [20]:
qbs_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 769 entries, 0 to 768
Data columns (total 14 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   player_display_name           769 non-null    object 
 1   recent_team                   769 non-null    object 
 2   season                        769 non-null    int64  
 3   opponent_team                 769 non-null    object 
 4   completion_percentage         693 non-null    float64
 5   yards_per_attempt             693 non-null    float64
 6   pass_tds_per_attempt          693 non-null    float64
 7   season_passing_yards          769 non-null    int64  
 8   season_passing_tds            769 non-null    int64  
 9   season_pass_attempts          769 non-null    int64  
 10  season_completions            769 non-null    int64  
 11  season_completion_percentage  737 non-null    float64
 12  season_yards_per_attempt      737 non-null    float64
 13  seaso

clean nan

In [21]:
qbs_df = qbs_df.dropna(subset=['completion_percentage'])

In [22]:
qbs_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 693 entries, 0 to 768
Data columns (total 14 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   player_display_name           693 non-null    object 
 1   recent_team                   693 non-null    object 
 2   season                        693 non-null    int64  
 3   opponent_team                 693 non-null    object 
 4   completion_percentage         693 non-null    float64
 5   yards_per_attempt             693 non-null    float64
 6   pass_tds_per_attempt          693 non-null    float64
 7   season_passing_yards          693 non-null    int64  
 8   season_passing_tds            693 non-null    int64  
 9   season_pass_attempts          693 non-null    int64  
 10  season_completions            693 non-null    int64  
 11  season_completion_percentage  693 non-null    float64
 12  season_yards_per_attempt      693 non-null    float64
 13  seaso

In [23]:
rushers_df = rushers_df.drop(columns= ['player_id',
                                       'player_name',
                                       'position', 
                                       'position_group',
                                       'headshot_url',
                                       'season_type',
                                       'passing_epa',
                                       'pacr',
                                       'dakota',
                                       'completions',
                                       'attempts',
                                       'passing_yards',
                                       'passing_tds',
                                       'interceptions',
                                       'sacks',
                                       'sack_yards',
                                       'sack_fumbles',
                                       'sack_fumbles_lost',
                                       'passing_air_yards',
                                       'passing_yards_after_catch',
                                       'passing_first_downs',
                                       'passing_epa',
                                       'passing_2pt_conversions',
                                       'rushing_epa',
                                       'receptions',
                                       'targets',
                                       'receiving_yards',
                                       'receiving_tds',
                                       'receiving_fumbles',
                                       'receiving_fumbles_lost',
                                       'receiving_air_yards',
                                       'receiving_yards_after_catch',
                                       'receiving_first_downs',
                                       'receiving_epa',
                                       'receiving_2pt_conversions',
                                       'racr',
                                       'target_share',
                                       'air_yards_share',
                                       'wopr',
                                       'special_teams_tds',
                                       'fantasy_points',
                                       'fantasy_points_ppr'
                                      ]).reset_index(drop=True)

rushers_df.head()

,player_display_name,recent_team,season,week,opponent_team,carries,rushing_yards,rushing_tds,rushing_fumbles,rushing_fumbles_lost,rushing_first_downs,rushing_2pt_conversions,yards_per_carry,tds_per_carry,season_rushing_yards,season_carries,season_rushing_tds,season_yards_per_carry,season_tds_per_carry
0,Steven Jackson,ATL,2014,1,NO,12,52,0,0,0,2,0,4.333333,0.000000,707,190,6,3.633017,0.027055
1,Steven Jackson,ATL,2014,2,CIN,11,46,0,0,0,2,0,4.181818,0.000000,707,190,6,3.633017,0.027055
2,Steven Jackson,ATL,2014,3,TB,14,54,1,0,0,5,0,3.857143,0.071429,707,190,6,3.633017,0.027055
3,Steven Jackson,ATL,2014,4,MIN,13,49,0,0,0,3,0,3.769231,0.000000,707,190,6,3.633017,0.027055
4,Steven Jackson,ATL,2014,5,NYG,13,37,1,0,0,3,0,2.846154,0.076923,707,190,6,3.633017,0.027055


In [24]:
rushers_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14451 entries, 0 to 14450
Data columns (total 19 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   player_display_name      14451 non-null  object 
 1   recent_team              14451 non-null  object 
 2   season                   14451 non-null  int64  
 3   week                     14451 non-null  int64  
 4   opponent_team            14451 non-null  object 
 5   carries                  14451 non-null  int64  
 6   rushing_yards            14451 non-null  int64  
 7   rushing_tds              14451 non-null  int64  
 8   rushing_fumbles          14451 non-null  int64  
 9   rushing_fumbles_lost     14451 non-null  int64  
 10  rushing_first_downs      14451 non-null  int64  
 11  rushing_2pt_conversions  14451 non-null  int64  
 12  yards_per_carry          12959 non-null  float64
 13  tds_per_carry            12959 non-null  float64
 14  season_rushing_yards  

drop nan

In [25]:
receivers_df = receivers_df.drop(columns= ['player_id',
                                           'player_name',
                                           'position', 
                                           'position_group',
                                           'headshot_url',
                                           'season_type',
                                           'passing_epa',
                                           'pacr',
                                           'dakota',
                                           'completions',
                                           'attempts',
                                           'passing_yards',
                                           'passing_tds',
                                           'interceptions',
                                           'sacks',
                                           'sack_yards',
                                           'sack_fumbles',
                                           'sack_fumbles_lost',
                                           'passing_air_yards',
                                           'passing_yards_after_catch',
                                           'passing_first_downs',
                                           'passing_epa',
                                           'passing_2pt_conversions',
                                           'carries',
                                           'rushing_yards',
                                           'rushing_tds',
                                           'rushing_fumbles',
                                           'rushing_fumbles_lost',
                                           'rushing_first_downs',
                                           'rushing_epa',
                                           'rushing_2pt_conversions',
                                           'receiving_epa',
                                           'racr',
                                           'target_share',
                                           'air_yards_share',
                                           'wopr',
                                           'special_teams_tds',
                                           'fantasy_points',
                                           'fantasy_points_ppr'
                                          ]).reset_index(drop=True)

receivers_df.head()

,player_display_name,recent_team,season,week,opponent_team,receptions,targets,receiving_yards,receiving_tds,receiving_fumbles,receiving_fumbles_lost,receiving_air_yards,receiving_yards_after_catch,receiving_first_downs,receiving_2pt_conversions,yards_per_reception,tds_per_reception,season_receiving_yards,season_receiving_tds,season_receptions,season_yards_per_reception,season_tds_per_reception
0,Steve Smith,BAL,2014,1,CIN,7,15,118,1,0,0,145,51,3,0,16.857143,0.142857,1065,6,79,12.944643,0.063839
1,Steve Smith,BAL,2014,2,PIT,6,10,71,0,0,0,87,45,4,0,11.833333,0.000000,1065,6,79,12.944643,0.063839
2,Steve Smith,BAL,2014,3,CLE,5,7,101,0,0,0,95,25,3,0,20.200000,0.000000,1065,6,79,12.944643,0.063839
3,Steve Smith,BAL,2014,4,CAR,7,10,139,2,0,0,102,69,6,0,19.857143,0.285714,1065,6,79,12.944643,0.063839
4,Steve Smith,BAL,2014,5,IND,5,7,34,0,1,1,42,7,1,0,6.800000,0.000000,1065,6,79,12.944643,0.063839


In [26]:
receivers_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30802 entries, 0 to 30801
Data columns (total 22 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   player_display_name          30802 non-null  object 
 1   recent_team                  30802 non-null  object 
 2   season                       30802 non-null  int64  
 3   week                         30802 non-null  int64  
 4   opponent_team                30802 non-null  object 
 5   receptions                   30802 non-null  int64  
 6   targets                      30802 non-null  int64  
 7   receiving_yards              30802 non-null  int64  
 8   receiving_tds                30802 non-null  int64  
 9   receiving_fumbles            30802 non-null  int64  
 10  receiving_fumbles_lost       30802 non-null  int64  
 11  receiving_air_yards          30802 non-null  int64  
 12  receiving_yards_after_catch  30802 non-null  int64  
 13  receiving_first_

## Qbs

In [27]:
qbs_df = qbs_df[qbs_df['season_pass_attempts'] >= 100]
qbs_df.drop(columns=['recent_team', 'opponent_team', 'completion_percentage', 'yards_per_attempt', 'pass_tds_per_attempt'], inplace=True)

In [28]:
qbs_df.head()

,player_display_name,season,season_passing_yards,season_passing_tds,season_pass_attempts,season_completions,season_completion_percentage,season_yards_per_attempt,season_pass_tds_per_attempt
0,AJ McCarron,2015,854,6,119,79,0.721109,6.477734,0.035461
6,Aaron Rodgers,2014,4381,38,520,341,0.669568,8.641434,0.082370
7,Aaron Rodgers,2015,3821,31,572,347,0.615191,6.733264,0.055985
8,Aaron Rodgers,2016,4428,40,610,401,0.659904,7.421580,0.069920
9,Aaron Rodgers,2017,1675,16,238,154,0.631227,6.738127,0.065600


In [29]:
qbs_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 428 entries, 0 to 768
Data columns (total 9 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   player_display_name           428 non-null    object 
 1   season                        428 non-null    int64  
 2   season_passing_yards          428 non-null    int64  
 3   season_passing_tds            428 non-null    int64  
 4   season_pass_attempts          428 non-null    int64  
 5   season_completions            428 non-null    int64  
 6   season_completion_percentage  428 non-null    float64
 7   season_yards_per_attempt      428 non-null    float64
 8   season_pass_tds_per_attempt   428 non-null    float64
dtypes: float64(3), int64(5), object(1)
memory usage: 33.4+ KB


## Rushers

In [30]:
rushers_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14451 entries, 0 to 14450
Data columns (total 19 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   player_display_name      14451 non-null  object 
 1   recent_team              14451 non-null  object 
 2   season                   14451 non-null  int64  
 3   week                     14451 non-null  int64  
 4   opponent_team            14451 non-null  object 
 5   carries                  14451 non-null  int64  
 6   rushing_yards            14451 non-null  int64  
 7   rushing_tds              14451 non-null  int64  
 8   rushing_fumbles          14451 non-null  int64  
 9   rushing_fumbles_lost     14451 non-null  int64  
 10  rushing_first_downs      14451 non-null  int64  
 11  rushing_2pt_conversions  14451 non-null  int64  
 12  yards_per_carry          12959 non-null  float64
 13  tds_per_carry            12959 non-null  float64
 14  season_rushing_yards  

In [31]:
# Get the index of the maximum week for each player
max_week_idx = rushers_df.groupby(['player_display_name', 'season'])['week'].idxmax()

# Select the rows corresponding to these indices
rushers_df = rushers_df.loc[max_week_idx].reset_index(drop=True)
rushers_df = rushers_df.drop(columns= 'week')
rushers_df.head()

,player_display_name,recent_team,season,opponent_team,carries,rushing_yards,rushing_tds,rushing_fumbles,rushing_fumbles_lost,rushing_first_downs,rushing_2pt_conversions,yards_per_carry,tds_per_carry,season_rushing_yards,season_carries,season_rushing_tds,season_yards_per_carry,season_tds_per_carry
0,A.J. Dillon,GB,2020,CHI,1,3,0,0,0,0,0,3.000000,0.0,242,46,2,5.878307,0.010582
1,A.J. Dillon,GB,2021,DET,14,63,0,0,0,3,0,4.500000,0.0,803,187,5,4.195966,0.022409
2,A.J. Dillon,GB,2022,DET,9,33,0,0,0,3,0,3.666667,0.0,770,186,7,4.165845,0.037448
3,A.J. Dillon,GB,2023,MIN,7,27,0,0,0,1,0,3.857143,0.0,613,178,2,3.370240,0.012857
4,Aaron Jones,GB,2017,MIN,3,13,0,0,0,0,0,4.333333,0.0,448,81,4,6.710791,0.118838


In [32]:
rushers_df = rushers_df[rushers_df['season_carries'] >= 100]
rushers_df.drop(columns=['recent_team', 'opponent_team', 'carries', 'rushing_yards', 'rushing_tds', 'rushing_fumbles', 'rushing_fumbles_lost', 'rushing_first_downs', 'rushing_2pt_conversions', 'yards_per_carry', 'tds_per_carry'], inplace=True)

In [33]:
rushers_df.head()

,player_display_name,season,season_rushing_yards,season_carries,season_rushing_tds,season_yards_per_carry,season_tds_per_carry
1,A.J. Dillon,2021,803,187,5,4.195966,0.022409
2,A.J. Dillon,2022,770,186,7,4.165845,0.037448
3,A.J. Dillon,2023,613,178,2,3.370240,0.012857
5,Aaron Jones,2018,728,133,8,5.305956,0.050587
6,Aaron Jones,2019,1084,236,16,4.327544,0.066562


In [34]:
rushers_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 455 entries, 1 to 1600
Data columns (total 7 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   player_display_name     455 non-null    object 
 1   season                  455 non-null    int64  
 2   season_rushing_yards    455 non-null    int64  
 3   season_carries          455 non-null    int64  
 4   season_rushing_tds      455 non-null    int64  
 5   season_yards_per_carry  455 non-null    float64
 6   season_tds_per_carry    455 non-null    float64
dtypes: float64(2), int64(4), object(1)
memory usage: 28.4+ KB


## Receivers

In [35]:
receivers_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30802 entries, 0 to 30801
Data columns (total 22 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   player_display_name          30802 non-null  object 
 1   recent_team                  30802 non-null  object 
 2   season                       30802 non-null  int64  
 3   week                         30802 non-null  int64  
 4   opponent_team                30802 non-null  object 
 5   receptions                   30802 non-null  int64  
 6   targets                      30802 non-null  int64  
 7   receiving_yards              30802 non-null  int64  
 8   receiving_tds                30802 non-null  int64  
 9   receiving_fumbles            30802 non-null  int64  
 10  receiving_fumbles_lost       30802 non-null  int64  
 11  receiving_air_yards          30802 non-null  int64  
 12  receiving_yards_after_catch  30802 non-null  int64  
 13  receiving_first_

In [36]:
# Get the index of the maximum week for each player
max_week_idx = receivers_df.groupby(['player_display_name', 'season'])['week'].idxmax()

# Select the rows corresponding to these indices
receivers_df = receivers_df.loc[max_week_idx].reset_index(drop=True)
receivers_df = receivers_df.drop(columns= 'week')
receivers_df.head()

,player_display_name,recent_team,season,opponent_team,receptions,targets,receiving_yards,receiving_tds,receiving_fumbles,receiving_fumbles_lost,receiving_air_yards,receiving_yards_after_catch,receiving_first_downs,receiving_2pt_conversions,yards_per_reception,tds_per_reception,season_receiving_yards,season_receiving_tds,season_receptions,season_yards_per_reception,season_tds_per_reception
0,A.J. Brown,TEN,2019,HOU,4,8,124,1,0,0,145,53,3,0,31.00,0.25,1051,8,52,19.626042,0.136979
1,A.J. Brown,TEN,2020,HOU,10,11,151,1,0,0,123,39,7,0,15.10,0.10,1075,11,70,15.804592,0.153741
2,A.J. Brown,TEN,2021,HOU,4,6,68,1,0,0,62,25,4,0,17.00,0.25,869,5,63,14.088068,0.067992
3,A.J. Brown,PHI,2022,NYG,4,10,95,0,0,0,156,34,4,0,23.75,0.00,1496,11,88,16.368791,0.126471
4,A.J. Brown,PHI,2023,NYG,1,1,9,0,1,1,8,1,0,0,9.00,0.00,1456,7,106,12.863119,0.053828


In [37]:
receivers_df = receivers_df[receivers_df['season_receptions'] >= 75]
receivers_df.drop(columns=['recent_team', 'opponent_team', 'receptions', 'targets', 'receiving_yards', 'receiving_tds', 'receiving_fumbles', 'receiving_fumbles_lost', 'receiving_air_yards', 'receiving_yards_after_catch', 'receiving_first_downs', 'receiving_2pt_conversions', 'yards_per_reception', 'tds_per_reception'], inplace=True)

In [38]:
receivers_df.head()

,player_display_name,season,season_receiving_yards,season_receiving_tds,season_receptions,season_yards_per_reception,season_tds_per_reception
3,A.J. Brown,2022,1496,11,88,16.368791,0.126471
4,A.J. Brown,2023,1456,7,106,12.863119,0.053828
9,A.J. Green,2015,1297,10,86,15.988014,0.110890
11,A.J. Green,2017,1078,8,75,13.566667,0.094494
25,Adam Humphries,2018,816,5,76,12.015923,0.055804


In [39]:
receivers_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 247 entries, 3 to 3321
Data columns (total 7 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   player_display_name         247 non-null    object 
 1   season                      247 non-null    int64  
 2   season_receiving_yards      247 non-null    int64  
 3   season_receiving_tds        247 non-null    int64  
 4   season_receptions           247 non-null    int64  
 5   season_yards_per_reception  247 non-null    float64
 6   season_tds_per_reception    247 non-null    float64
dtypes: float64(2), int64(4), object(1)
memory usage: 15.4+ KB


# Modeling

## Starting with Qbs

In [40]:
# Features (excluding the target and the player name)
X = qbs_df[['season', 'season_passing_yards', 'season_pass_attempts', 
            'season_completion_percentage', 'season_yards_per_attempt', 
            'season_pass_tds_per_attempt']]
# Target variable
y = qbs_df['season_passing_tds']

In [41]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

### Dummy

In [42]:
# Initialize the dummy model
dummy_model = DummyRegressor(strategy='mean')

# Train the model
dummy_model.fit(X_train, y_train)

# Make predictions
y_dummy_pred = dummy_model.predict(X_test)

# Calculate mean squared error
dummy_mse = mean_squared_error(y_test, y_dummy_pred)
print(f'Dummy Model Mean Squared Error: {dummy_mse}')

Dummy Model Mean Squared Error: 101.45507582138174


### Decision Tree

In [43]:
# Initialize the decision tree model
tree_model = DecisionTreeRegressor(random_state=42)

# Train the model
tree_model.fit(X_train, y_train)

# Make predictions
y_tree_pred = tree_model.predict(X_test)

# Calculate mean squared error
tree_mse = mean_squared_error(y_test, y_tree_pred)
print(f'Decision Tree Model Mean Squared Error: {tree_mse}')

Decision Tree Model Mean Squared Error: 5.546511627906977


In [44]:
# Get feature importances
feature_importances = tree_model.feature_importances_

# Create a DataFrame for better visualization
feature_importances_df = pd.DataFrame({
    'Feature': X.columns,
    'Importance': feature_importances
}).sort_values(by='Importance', ascending=False)

print(feature_importances_df)


                        Feature  Importance
1          season_passing_yards    0.774394
5   season_pass_tds_per_attempt    0.204841
2          season_pass_attempts    0.012473
4      season_yards_per_attempt    0.004374
3  season_completion_percentage    0.002337
0                        season    0.001581


FSM complete! hyper-parameter tuning and more models to come!

### Random Forest

In [45]:
# Correlation analysis
corr_matrix = qbs_df.corr()
print(corr_matrix['season_passing_tds'].sort_values(ascending=False))

season_passing_tds              1.000000
season_passing_yards            0.912945
season_completions              0.876831
season_pass_attempts            0.859070
season_pass_tds_per_attempt     0.757278
season_yards_per_attempt        0.558755
season_completion_percentage    0.477187
season                         -0.080066
Name: season_passing_tds, dtype: float64


In [46]:
# Cross-validation with RandomForest
rf_model = RandomForestRegressor(random_state=42)
cv_scores = cross_val_score(rf_model, X, y, cv=5, scoring='neg_mean_squared_error')
print(f'Cross-validated Mean Squared Error: {-cv_scores.mean()}')

Cross-validated Mean Squared Error: 2.9337659863201098


In [47]:
# Train the model on the entire dataset
rf_model.fit(X, y)
importances = rf_model.feature_importances_
feature_importances_df = pd.DataFrame({
    'Feature': X.columns,
    'Importance': importances
}).sort_values(by='Importance', ascending=False)
print(feature_importances_df)

                        Feature  Importance
1          season_passing_yards    0.763046
5   season_pass_tds_per_attempt    0.204351
2          season_pass_attempts    0.023923
4      season_yards_per_attempt    0.003422
3  season_completion_percentage    0.003130
0                        season    0.002128


the overwhelmingly most important thing to a QB getting the most touchdowns in a season is getting the most passing yards in a season therefore our reccomendations will be the top QB in passing yards and TDs

QB visuals

## Rushers

In [48]:
rushers_df.head()

,player_display_name,season,season_rushing_yards,season_carries,season_rushing_tds,season_yards_per_carry,season_tds_per_carry
1,A.J. Dillon,2021,803,187,5,4.195966,0.022409
2,A.J. Dillon,2022,770,186,7,4.165845,0.037448
3,A.J. Dillon,2023,613,178,2,3.370240,0.012857
5,Aaron Jones,2018,728,133,8,5.305956,0.050587
6,Aaron Jones,2019,1084,236,16,4.327544,0.066562


In [49]:
# Features (excluding the target and the player name)
X1 = rushers_df[['season', 'season_rushing_yards', 'season_carries', 
            'season_yards_per_carry', 'season_tds_per_carry']]
# Target variable
y1 = rushers_df['season_rushing_tds']

In [50]:
X1_train, X1_test, y1_train, y1_test = train_test_split(X1, y1, test_size=0.2, random_state=42)

### Dummy

In [51]:
# Initialize the dummy model
dummy1_model = DummyRegressor(strategy='mean')

# Train the model
dummy1_model.fit(X1_train, y1_train)

# Make predictions
y1_dummy_pred = dummy1_model.predict(X1_test)

# Calculate mean squared error
dummy1_mse = mean_squared_error(y1_test, y1_dummy_pred)
print(f'Dummy Model Mean Squared Error: {dummy1_mse}')

Dummy Model Mean Squared Error: 10.292423922231613


### Decision Tree

In [52]:
# Initialize the decision tree model
tree_model1 = DecisionTreeRegressor(random_state=42)

# Train the model
tree_model1.fit(X1_train, y1_train)

# Make predictions
y1_tree_pred = tree_model1.predict(X1_test)

# Calculate mean squared error
tree_mse1 = mean_squared_error(y1_test, y1_tree_pred)
print(f'Decision Tree Model Mean Squared Error: {tree_mse1}')

Decision Tree Model Mean Squared Error: 1.9230769230769231


In [53]:
# Get feature importances
feature_importances1 = tree_model1.feature_importances_

# Create a DataFrame for better visualization
feature_importances1_df = pd.DataFrame({
    'Feature': X1.columns,
    'Importance': feature_importances1
}).sort_values(by='Importance', ascending=False)

print(feature_importances1_df)


                  Feature  Importance
4    season_tds_per_carry    0.650189
1    season_rushing_yards    0.239607
2          season_carries    0.079863
3  season_yards_per_carry    0.024090
0                  season    0.006251


### Random Forest

In [54]:
# Correlation analysis
corr_matrix1 = rushers_df.corr()
print(corr_matrix1['season_rushing_tds'].sort_values(ascending=False))

season_rushing_tds        1.000000
season_tds_per_carry      0.752336
season_rushing_yards      0.681510
season_carries            0.628037
season_yards_per_carry    0.259760
season                    0.070169
Name: season_rushing_tds, dtype: float64


In [55]:
# Cross-validation with RandomForest
rf_model1 = RandomForestRegressor(random_state=42)
cv_scores1 = cross_val_score(rf_model1, X1, y1, cv=5, scoring='neg_mean_squared_error')
print(f'Cross-validated Mean Squared Error: {-cv_scores1.mean()}')

Cross-validated Mean Squared Error: 1.1815000000000002


In [56]:
# Train the model on the entire dataset
rf_model1.fit(X1, y1)
importances1 = rf_model1.feature_importances_
feature_importances1_df = pd.DataFrame({
    'Feature': X1.columns,
    'Importance': importances1
}).sort_values(by='Importance', ascending=False)
print(feature_importances1_df)

                  Feature  Importance
4    season_tds_per_carry    0.683756
2          season_carries    0.178107
1    season_rushing_yards    0.113704
3  season_yards_per_carry    0.018153
0                  season    0.006281


season tds and tds per carry no suprise there

## Receivers

In [57]:
receivers_df.head()

,player_display_name,season,season_receiving_yards,season_receiving_tds,season_receptions,season_yards_per_reception,season_tds_per_reception
3,A.J. Brown,2022,1496,11,88,16.368791,0.126471
4,A.J. Brown,2023,1456,7,106,12.863119,0.053828
9,A.J. Green,2015,1297,10,86,15.988014,0.110890
11,A.J. Green,2017,1078,8,75,13.566667,0.094494
25,Adam Humphries,2018,816,5,76,12.015923,0.055804


In [58]:
# Features (excluding the target and the player name)
X2 = receivers_df[['season', 'season_receiving_yards', 'season_receptions', 
            'season_yards_per_reception', 'season_tds_per_reception']]
# Target variable
y2 = receivers_df['season_receiving_tds']

In [59]:
X2_train, X2_test, y2_train, y2_test = train_test_split(X2, y2, test_size=0.2, random_state=42)

### Dummy

In [60]:
# Initialize the dummy model
dummy2_model = DummyRegressor(strategy='mean')

# Train the model
dummy2_model.fit(X2_train, y2_train)

# Make predictions
y2_dummy_pred = dummy2_model.predict(X2_test)

# Calculate mean squared error
dummy2_mse = mean_squared_error(y2_test, y2_dummy_pred)
print(f'Dummy Model Mean Squared Error: {dummy2_mse}')

Dummy Model Mean Squared Error: 9.426912829498313


### Decision Tree

In [61]:
# Initialize the decision tree model
tree_model2 = DecisionTreeRegressor(random_state=42)

# Train the model
tree_model2.fit(X2_train, y2_train)

# Make predictions
y2_tree_pred = tree_model2.predict(X2_test)

# Calculate mean squared error
tree_mse2 = mean_squared_error(y2_test, y2_tree_pred)
print(f'Decision Tree Model Mean Squared Error: {tree_mse2}')

Decision Tree Model Mean Squared Error: 3.34


In [62]:
# Get feature importances
feature_importances2 = tree_model2.feature_importances_

# Create a DataFrame for better visualization
feature_importances2_df = pd.DataFrame({
    'Feature': X2.columns,
    'Importance': feature_importances2
}).sort_values(by='Importance', ascending=False)

print(feature_importances2_df)

                      Feature  Importance
4    season_tds_per_reception    0.762494
1      season_receiving_yards    0.125304
2           season_receptions    0.076144
3  season_yards_per_reception    0.028178
0                      season    0.007880


### Random Forest

In [63]:
# Correlation analysis
corr_matrix2 = receivers_df.corr()
print(corr_matrix2['season_receiving_tds'].sort_values(ascending=False))

season_receiving_tds          1.000000
season_tds_per_reception      0.856370
season_receiving_yards        0.531732
season_yards_per_reception    0.352271
season_receptions             0.330012
season                       -0.100267
Name: season_receiving_tds, dtype: float64


In [64]:
# Cross-validation with RandomForest
rf_model2 = RandomForestRegressor(random_state=42)
cv_scores2 = cross_val_score(rf_model2, X2, y2, cv=5, scoring='neg_mean_squared_error')
print(f'Cross-validated Mean Squared Error: {-cv_scores2.mean()}')

Cross-validated Mean Squared Error: 1.7081099020408164


In [65]:
# Train the model on the entire dataset
rf_model2.fit(X2, y2)
importances2 = rf_model2.feature_importances_
feature_importances2_df = pd.DataFrame({
    'Feature': X2.columns,
    'Importance': importances2
}).sort_values(by='Importance', ascending=False)
print(feature_importances2_df)

                      Feature  Importance
4    season_tds_per_reception    0.782393
1      season_receiving_yards    0.100856
2           season_receptions    0.076440
3  season_yards_per_reception    0.027769
0                      season    0.012543
